<a href="https://colab.research.google.com/github/abedyk/dw_matrix_car/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np


import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance


In [8]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [13]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
	if isinstance(df[feat][0], list): continue

	factorized_values = df[feat].factorize()[0]
	if SUFFIX_CAT in feat:
	  df[feat] = factorized_values
	else:
	  df[feat + SUFFIX_CAT] = factorized_values

In [15]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [16]:
X = df[cat_feats].values
y = df['price_value'].values


model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

NameError: ignored

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [19]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats )

[09:19:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:20:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [22]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats )

[09:27:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:27:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [23]:
feats = ['param_napęd__cat', 'param_rok-produkcji__cat', 'param_stan__cat','param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat','param_pojemność-skokowa__cat', 'seller_name__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']
len(feats)
run_model(xgb.XGBRegressor(**xgb_params), feats )

[09:30:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:30:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:30:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [24]:
def obj_func(params):
    print("Training with params: ")
    print(params)

    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

    return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_params = {
    'learning_rate':	hp.choice('learning_rate',	np.arange(0.05, 0.31, 0.05)),
    'max_depth':	hp.choice('max_depth',		np.arange(5, 16, 1, dtype=int)),
    'subsample':	hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':	'reg:squarederror',
    'seed':0,
}

## run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.2, 'max_depth': 10, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.1, 'max_depth': 11, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 8, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.3, 'max_depth': 11, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.2, 'max_depth': 15, 'objective': 'reg:squarederror', 'seed': 0, 'subsample

{'colsample_bytree': 0.55,
 'learning_rate': 3,
 'max_depth': 5,
 'subsample': 1.0}